# Match Mentions to Variants

In [1]:
import sys
import os
import re
import json
import sqlite3
import functools
import itertools
import pandas as pd

In [2]:
os.environ['UTA_DB_URL']='postgresql://anonymous@rcurrie-uta:5432/uta/uta_20170117'
os.environ['HGVS_SEQREPO_DIR']='/home/jovyan/data/pubmunch/references/seqrepo/2018-10-03'

!pip install --user --quiet hgvs
import hgvs.parser
import hgvs.dataproviders.uta
import hgvs.assemblymapper

parser = hgvs.parser.Parser()
server = hgvs.dataproviders.uta.connect()
mapper = hgvs.assemblymapper.AssemblyMapper(server, assembly_name="GRCh38")

You are using pip version 18.1, however version 19.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## HGVS Mapping
Use the [HGVS](https://hgvs.readthedocs.io/en/stable/) library to parse and map using a local reference server.

In [3]:
@functools.lru_cache(maxsize=None)
def parse_and_map_hgvs(candidate):
    assert candidate
    # Normalize single deletions: NM_007300.3:c.1100C>None -> NM_007300.3:c.1100delC
    candidate = re.sub(r"(NM.*c\.\d*)([ATCGatcg]+)(>None)", r"\1del\2", candidate)

    # TODO: Normalize multiple deletions and delins
    # TODO: Limit to only specific BRCA transcripts
    # ex: 23199084	NM_007294.3:c.2681AA>None|NM_007300.3:c.2681AA>None

    try:
        parsed_hgvs = parser.parse_hgvs_variant(candidate)
        try:
            return str(mapper.c_to_g(parsed_hgvs))
        except hgvs.exceptions.HGVSInvalidVariantError:
            print("Failed Mapping: (HGVSInvalidVariantError): {}".format(candidate))
        except hgvs.exceptions.HGVSInvalidIntervalError:
            print("Failed Mapping (HGVSInvalidIntervalError): {}".format(candidate))
        except hgvs.exceptions.HGVSDataNotAvailableError: 
            print("Failed Mapping (HGVSDataNotAvailableError): {}".format(candidate))

    except hgvs.exceptions.HGVSParseError:
        print("Failed to parse: {}".format(candidate))

    return ""

parse_and_map_hgvs("NM_007294.3:c.1105G>A")

'NC_000017.11:g.43094426C>T'

In [4]:
# Switch to the crawl we want to match and export from
# os.chdir(os.path.expanduser("/home/jovyan/data/pubmunch/crawl-2018-11-14/"))
os.chdir(os.path.expanduser("/home/jovyan/data/pubmunch/crawl/"))

## Normalize BRCA Exchange Variants

Parse and map to genomic HGVS all of the variants tracked in the latest BRCA Exchange database

In [5]:
variants = pd.read_csv("output/release/built_with_change_types.tsv",
                       sep="\t", header=0,
                       usecols=["pyhgvs_Genomic_Coordinate_38", "pyhgvs_cDNA", "Synonyms"])
print("Loaded {} variants".format(variants.shape[0]))

Loaded 21695 variants


In [6]:
variants = variants.sort_values("pyhgvs_cDNA")

In [7]:
%%time
variants["norm_g_hgvs"] = variants["pyhgvs_cDNA"].apply(parse_and_map_hgvs)

Failed Mapping (HGVSInvalidIntervalError): NM_000059.3:c.-1017T>C
Failed Mapping (HGVSInvalidIntervalError): NM_000059.3:c.-1027C>A
Failed Mapping (HGVSInvalidIntervalError): NM_000059.3:c.-1122T>C
Failed Mapping (HGVSInvalidIntervalError): NM_000059.3:c.-1193C>T
Failed Mapping (HGVSInvalidIntervalError): NM_000059.3:c.-251G>A
Failed Mapping (HGVSInvalidIntervalError): NM_000059.3:c.-268C>G
Failed Mapping (HGVSInvalidIntervalError): NM_000059.3:c.-280delG
Failed Mapping (HGVSInvalidIntervalError): NM_000059.3:c.-296C>T
Failed Mapping (HGVSInvalidIntervalError): NM_000059.3:c.-357T>C
Failed Mapping (HGVSInvalidIntervalError): NM_000059.3:c.-395C>T
Failed Mapping (HGVSInvalidIntervalError): NM_000059.3:c.-407G>A
Failed Mapping (HGVSInvalidIntervalError): NM_000059.3:c.-418A>G
Failed Mapping (HGVSInvalidIntervalError): NM_000059.3:c.-481G>A
Failed Mapping (HGVSInvalidIntervalError): NM_000059.3:c.-495G>A
Failed Mapping (HGVSInvalidIntervalError): NM_000059.3:c.-521delG
Failed Mapping (HGV

In [8]:
# Load output from match.py
# variants = pd.read_csv("variants-normalized.tsv",
#                        sep="\t", header=0, index_col="norm_g_hgvs")

## Normalize and Match Mentions to Variants

In [38]:
# Wrangle mentions
mentions = pd.read_csv("mutations-trimmed.tsv", sep="\t", header=0, dtype="str")
print(mentions.shape[0])
mentions = mentions.fillna("")
print(mentions.shape[0])
mentions = mentions[mentions.mutSnippets != ""]
print(mentions.shape[0])
mentions = mentions[(mentions.hgvsCoding != "") | (mentions.texts != "")]
print(mentions.shape[0])

300
300
271
271


In [39]:
mentions.docId.dtype

dtype('O')

In [10]:
mentions = mentions.sort_values(["docId", "hgvsCoding"])

In [41]:
def next_mention():
    for i, row in mentions.iterrows():
        matched = False
        norm_g_hgvs = ""
    
        for raw_hgvs in set([r.strip() for r in row.hgvsCoding.split("|")]):           

            if not raw_hgvs:
                continue
                
            norm_g_hgvs = parse_and_map_hgvs(raw_hgvs)
            
            if not norm_g_hgvs:
                continue
            
            # normalized to normalized
            if norm_g_hgvs in variants:
                matched = True
                yield (variants[norm_g_hgvs].pyhgvs_Genomic_Coordinate_38, row.docId, row.mutSnippets, 1)

            # normalized to synonym (BRCA Exchange synonyms replace : with .)
            for i, hit in variants.loc[
                variants.Synonyms.str.contains(norm_g_hgvs.replace(":", "."), regex=False)].iterrows():
                matched = True
                yield (hit.pyhgvs_Genomic_Coordinate_38, row.docId, row.mutSnippets, 2)

        # texts to synonym
        # Note: Could always run but adds 40+ per variant...so only run if nothing else works
        if not matched:
            for text in set([t.strip() for t in row.texts.split("|")]):
                for i, hit in variants[variants.Synonyms.str.contains(text, regex=False)].iterrows():
#                 for i, hit in itertools.islice(
#                     variants[variants.Synonyms.str.contains(text, regex=False)].iterrows(), 10):
                    matched = True
                    yield (hit.pyhgvs_Genomic_Coordinate_38, row.docId, row.mutSnippets, 3)

        if not matched:
            print("Failed to match: hgvsCoding={} Mapped={} Texts={}".format(
                raw_hgvs, norm_g_hgvs, row.texts))    
                

# For each mention try to parse and normalize the hgvs
matches = pd.DataFrame(
    [m for m in next_mention()],
    columns=["pyhgvs_Genomic_Coordinate_38", "pmid", "snippets", "score"]
)

Failed to match: hgvsCoding=NM_000410.3:c.845G>A Mapped=NC_000006.12:g.26092913G>A Texts= C282Y|C282Y
Failed to match: hgvsCoding=NM_001257195.1:c.4T>G Mapped=NC_000005.10:g.137698400A>C Texts=S2A
Failed to match: hgvsCoding= Mapped= Texts= L2H
Failed to match: hgvsCoding= Mapped= Texts=S2A
Failed to match: hgvsCoding= Mapped= Texts=2294delG
Failed to match: hgvsCoding= Mapped= Texts=C2457T
Failed to match: hgvsCoding= Mapped= Texts=G1639T
Failed to match: hgvsCoding= Mapped= Texts=G5255A
Failed to match: hgvsCoding= Mapped= Texts=C4731T
Failed to match: hgvsCoding= Mapped= Texts=2294delG
Failed to match: hgvsCoding= Mapped= Texts= C2457T
Failed to match: hgvsCoding= Mapped= Texts=G1639T
Failed to match: hgvsCoding= Mapped= Texts=G5255A
Failed to match: hgvsCoding= Mapped= Texts=C4731T
Failed to match: hgvsCoding= Mapped= Texts=/5382insC
Failed to match: hgvsCoding= Mapped= Texts=S1C
Failed to match: hgvsCoding= Mapped= Texts=S1C
Failed to match: hgvsCoding= Mapped= Texts=I92P
Failed t

## Prune

In [ ]:
print("Initial # Matches", matches.shape[0])
pruned_matches = matches.drop_duplicates(["pyhgvs_Genomic_Coordinate_38", "pmid", "snippets"])
print("After dropping duplicates of pyhgvs_Genomic_Coordinate_38+pmid+snippets: {}".format(pruned_matches.shape[0]))
# print("Total unique genomic hgvs variants: {}".format(mentions.index.unique().shape[0]))
# mentions.head()

In [ ]:
# Some of the snippets are multiple mentions separated by | so unpack these,
# but limit to 3 as some have as many as 168!
print("Unpacking {} of {} snippets with multiple phrases seaparated by '|'".format(
    pruned_matches[pruned_matches.snippets.str.contains("|", regex=False)].shape[0], pruned_matches.shape[0]))

# https://stackoverflow.com/questions/17116814/pandas-how-do-i-split-text-in-a-column-into-multiple-rows/21032532

# Reset index so each row id is unique vs. norm_g_hgvs
df = pruned_matches.reset_index()
# df = variant_mentions[variant_mentions.snippet.str.contains("|", regex=False)].iloc[0:100].reset_index()

# Generate a new dataframe splitting each snippet segment into its own row
# Limit to max of 3 as some of them have > 100
snippets = df.apply(lambda x: pd.Series(x.snippets.split("|")[:3]), axis=1).stack()

# To line up with the original index
snippets.index = snippets.index.droplevel(-1)

# Join back to the original dataframe replaceing the old "snippet" columnd
snippets.name = "snippets"
del df["snippets"]
exploded = df.join(snippets).drop_duplicates(["pyhgvs_Genomic_Coordinate_38", "snippets"]).set_index("pyhgvs_Genomic_Coordinate_38")
print("{} individual snippets after expanding and de-duplicating snippets".format(exploded.shape[0]))

exploded.head()

## Export

In [ ]:
connection = sqlite3.connect("file:text/articles.db?mode=ro", uri=True)
articles = pd.read_sql_query("SELECT * FROM articles", connection)
articles.pmid = articles.pmid.astype(str)
print("{} articles loaded from the articles sqlite database".format(articles.shape[0]))
articles.head()

In [ ]:
# Variants by pyhgvs_Genomic_Coordinate_38 by pmid with all snippets in a list
combined = exploded.groupby(["pyhgvs_Genomic_Coordinate_38", "pmid"])["snippets"].apply(lambda s: s.tolist())
print("Combined {} separate snippets down to {} after grouping by pmid".format(exploded.shape[0], combined.shape[0]))

In [ ]:
literature = {
    "date": open("date.txt").read().strip(),
    "papers": articles[articles.pmid.isin(matches.pmid)].set_index("pmid", drop=False).to_dict(orient="index"),
    "variants": {
        k: {kk: vv[0] for kk, vv in v.unstack().transpose().iterrows()}
        for k, v in combined.groupby("pyhgvs_Genomic_Coordinate_38")},
}

with open("literature.json", "w") as output:
    output.write(json.dumps(literature, sort_keys=True))
    
print("Exported {} variants in {} papers".format(
    len(literature["variants"].keys()), len(literature["papers"].keys())))